In [20]:
import pandas as pd
from datetime import date, timedelta
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [63]:
from mainDriver_NEW import austinBikeShareVsCrime

Initialize Class

In [64]:
test = austinBikeShareVsCrime()

## Read and Format Data

In [65]:
test.readAndFormatData()

c:\users\jorda\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Make sure all data has been read in correctly

In [66]:
test.austin_bikeshare_stations_df.head()

,latitude,location,longitude,name,station_id,status
0,30.27041,(30.27041 -97.75046),-97.75046,West & 6th St.,2537,active
1,30.26452,(30.26452 -97.7712),-97.77120,Barton Springs Pool,2572,active
2,30.27595,(30.27595 -97.74739),-97.74739,ACC - Rio Grande & 12th,2545,closed
3,30.28480,(30.2848 -97.72756),-97.72756,Red River & LBJ Library,1004,closed
4,30.26694,(30.26694 -97.74939),-97.74939,Nueces @ 3rd,1008,moved


In [67]:
test.austin_bikeshare_trips_df.head()

,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,subscriber_type,trip_id,year
0,8.0,19:12:00,41,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,9900082882,2015.0
1,141.0,2:06:04,6,2570.0,South Congress & Academy,10.0,2494.0,2nd & Congress,2016-10-30 02:06:04,Local365,12617682,2016.0
2,578.0,16:28:27,13,2498.0,Convention Center / 4th St. @ MetroRail,3.0,2538.0,Bullock Museum @ Congress & MLK,2016-03-11 16:28:27,Local365,9075366,2016.0
3,555.0,15:12:00,80,2712.0,Toomey Rd @ South Lamar,11.0,2497.0,Capitol Station / Congress & 11th,2014-11-23 15:12:00,24-Hour Kiosk (Austin B-cycle),9900319298,2014.0
4,86.0,15:39:13,25,3377.0,MoPac Pedestrian Bridge @ Veterans Drive,4.0,2707.0,Rainey St @ Cummings,2017-04-16 15:39:13,Walk Up,14468597,2017.0


In [68]:
test.crimeWDistance.head()

,Unnamed: 0,Incident Number,Highest Offense Description,Highest Offense Code,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,Location Type,Address,Zip Code,Council District,APD Sector,APD District,PRA,Census Tract,Clearance Status,Clearance Date,UCR Category,Category Description,X-coordinate,Y-coordinate,Latitude,Longitude,Location,year,latitude,longitude,distance to 1001,distance to 1002,distance to 1003,distance to 1004,distance to 1005,distance to 1006,distance to 1007,distance to 1008,distance to 2494,distance to 2495,distance to 2496,distance to 2497,distance to 2498,distance to 2499,distance to 2500,distance to 2501,distance to 2502,distance to 2503,distance to 2504,distance to 2536,distance to 2537,distance to 2538,distance to 2539,distance to 2540,distance to 2541,distance to 2542,distance to 2544,distance to 2545,distance to 2546,distance to 2547,distance to 2548,distance to 2549,distance to 2550,distance to 2552,distance to 2561,distance to 2562,distance to 2563,distance to 2564,distance to 2565,distance to 2566,distance to 2567,distance to 2568,distance to 2569,distance to 2570,distance to 2571,distance to 2572,distance to 2574,distance to 2575,distance to 2576,distance to 2707,distance to 2711,distance to 2712,distance to 2822,distance to 2823,distance to 3291,distance to 3292,distance to 3293,distance to 3294,distance to 3377,distance to 3381,distance to 3390,distance to 3455,distance to 3464,distance to 3513,distance to 3619,distance to 3621,distance to 3635,distance to 3660,distance to 3684,distance to 3685,distance to 3686,distance to 3687
0,2,2.017257e+10,AGG ROBBERY/DEADLY WEAPON,300,N,9/14/2017 22:37,2017-09-14,2237,9/14/2017 22:37,9/14/2017,2237,STREETS / HWY / ROAD / ALLEY,E ST JOHNS AVE / ROLAND JOHNSON DR,78752.0,4.0,ID,8,279.0,18.04,O,9/20/2017,120,Robbery,3125442,3125442,30.333542,-97.705762,"(30.33354183, -97.70576196)",2017,30.333542,-97.705762,7604.400028,8056.778502,7932.173791,5809.766825,6765.104832,9624.132589,8127.289671,8508.187862,8533.389417,8316.266847,7889.640442,7585.522699,8280.013504,8602.470381,8388.677230,8449.747617,9287.978352,10080.712560,10332.288670,8024.107872,8227.610802,6675.474769,8384.438535,6946.627875,7209.576674,8204.306461,8339.001917,7548.557058,7568.514346,6518.924170,6340.644607,9260.220808,8315.602223,8546.931117,7374.952232,7875.920024,8739.286665,8142.667034,8034.813075,8756.021182,9400.566119,7503.677136,7495.071925,9927.584041,7805.625708,9918.314681,9619.129702,9103.567927,8986.952307,9243.320538,9570.349247,9318.846229,8240.350199,8598.853097,7539.053881,8371.600198,8749.425021,8148.711803,9012.834723,8159.926571,8103.097122,8388.124183,7332.450213,9203.059794,8089.237028,8500.560334,7403.668088,8064.106508,8672.877964,8165.084092,9528.506312,9466.067193
1,6,2.014505e+10,BURGLARY OF VEHICLE,601,N,10/11/2014 17:00,2014-10-11,1700,10/12/2014 4:13,10/12/2014,413,PARKING LOTS / GARAGE,E 7TH ST / TRINITY ST,78701.0,9.0,GE,2,382.0,11.00,N,11/4/2014,23F,Theft,3115529,3115529,30.268006,-97.738955,"(30.26800598, -97.73895531)",2014,30.268006,-97.738955,439.486675,1094.093681,725.402333,2164.405511,1242.418015,2824.355216,612.457236,1009.102401,620.718543,499.009584,342.942902,557.101152,353.178943,833.654551,872.288642,1390.863694,1396.515295,2131.367055,2381.819686,896.410164,1136.734719,1379.545093,434.150885,1349.410738,981.108744,1283.083416,2533.424726,1198.494511,1282.703875,1796.529693,1993.098782,1407.290930,765.473691,1242.415010,601.654545,151.305090,870.451223,722.631875,81.334712,1528.346307,1670.781105,1016.360500,810.624268,1978.124447,246.887547,3120.888198,2820.247966,1744.510534,1110.477264,1360.891291,2236.485482,1932.646079,2099.313929,3064.395837,438.150187,1719.479472,2558.881942,706.498163,3097.867394,2909.703617,253.575454,875.026581,1939.366975,1261.235632,374.788366,999.175581,1070.419417,799.939864,785.804796,1377.089822,2430.776166,1534.035945
2,9,2.015501e+10,FORGERY AND PASSING,1000,N,2

In [69]:
test.austin_weather_df.head()

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,SeaLevelPressureHighInches,SeaLevelPressureAvgInches,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events
0,2013-12-21,74,60,45,67,49,43,93,75,57,29.86,29.68,29.59,10,7,2,20,4,31,0.46,"Rain , Thunderstorm"
1,2013-12-22,56,48,39,43,36,28,93,68,43,30.41,30.13,29.87,10,10,5,16,6,25,0,
2,2013-12-23,58,45,32,31,27,23,76,52,27,30.56,30.49,30.41,10,10,10,8,3,12,0,
3,2013-12-24,61,46,31,36,28,21,89,56,22,30.56,30.45,30.3,10,10,7,12,4,20,0,
4,2013-12-25,58,50,41,44,40,36,86,71,56,30.41,30.33,30.27,10,10,7,10,2,16,T,


## Featue Creation

#### Day of Week

In [70]:
index = pd.date_range(test.INDEX_START_DATE, test.INDEX_END_DATE - timedelta(days=1), freq='d')
columns = []
featureDF = pd.DataFrame(index=index, columns=columns)
featureDF['date'] = index
featureDF

,date
2013-12-21,2013-12-21
2013-12-22,2013-12-22
2013-12-23,2013-12-23
2013-12-24,2013-12-24
2013-12-25,2013-12-25
...,...
2017-07-26,2017-07-26
2017-07-27,2017-07-27
2017-07-28,2017-07-28
2017-07-29,2017-07-29


In [71]:
feature = test.oneHotEncodingDayOfWeek(featureDF, 'date')
test.test.head()


AttributeError: 'austinBikeShareVsCrime' object has no attribute 'test'

In [58]:
def oneHotEncodingDayOfWeek(df, df_date_column):
    day_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

    def is_day(x, day_index):
        if x == day_index:
            return 1
        else:
            return 0

    dfCopy = df.copy()
    dfCopy['temp'] = pd.to_datetime(dfCopy[df_date_column]).dt.dayofweek
    for index, day in enumerate(day_of_week):
        dfCopy[f'is{day}'] = dfCopy['temp'].apply(lambda x: is_day(x, index))
    dfCopy = dfCopy.drop(columns=['temp'])
    return dfCopy

In [59]:
oneHotEncodingDayOfWeek(featureDF, 'date')

,date,isMonday,isTuesday,isWednesday,isThursday,isFriday,isSaturday,isSunday
2013-12-21,2013-12-21,0,0,0,0,0,1,0
2013-12-22,2013-12-22,0,0,0,0,0,0,1
2013-12-23,2013-12-23,1,0,0,0,0,0,0
2013-12-24,2013-12-24,0,1,0,0,0,0,0
2013-12-25,2013-12-25,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...
2017-07-26,2017-07-26,0,0,1,0,0,0,0
2017-07-27,2017-07-27,0,0,0,1,0,0,0
2017-07-28,2017-07-28,0,0,0,0,1,0,0
2017-07-29,2017-07-29,0,0,0,0,0,1,0
